# Calculate daily WBGTmean, WBGTmax, WBGThalf

In [1]:
%matplotlib inline

from pathlib import Path
from datetime import date

import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from numba import jit
from scipy import stats
from config import DATA_SRC, WEATHER_SRC

import weather_ecmwf

import climate_conversions


In [2]:
weather = weather_ecmwf.open_mfdataset(WEATHER_SRC / 'ecmwf' / 'daily_sfc',
                                       chunks={'time':100}
                                      )

# weather = weather_ecmwf.open_mfdataset(WEATHER_SRC / 'ecmwf' / 'daily_test',
#                                        chunks={'time':100}
#                                       )

In [3]:
weather

<xarray.Dataset>
Dimensions:               (latitude: 241, longitude: 480, time: 55520)
Coordinates:
  * longitude             (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 ...
  * latitude              (latitude) float32 90.0 89.25 88.5 87.75 87.0 ...
  * time                  (time) datetime64[ns] 1980-01-01 ...
Data variables:
    surface_pressure      (time, latitude, longitude) float32 dask.array<shape=(55520, 241, 480), chunksize=(100, 241, 480)>
    temperature_2m        (time, latitude, longitude) float32 dask.array<shape=(55520, 241, 480), chunksize=(100, 241, 480)>
    temperature_dewpoint  (time, latitude, longitude) float32 dask.array<shape=(55520, 241, 480), chunksize=(100, 241, 480)>
Attributes:
    Conventions:  CF-1.6
    history:      2018-03-28 21:52:42 GMT by grib_to_netcdf-2.7.0: grib_to_ne...

## Generate Relative Humidity

> We just calculate this in the process of calculating WBT

In [8]:
# relative_humidity = xr.apply_ufunc(climate_conversions.calculate_relative_humidity, 
#                                    weather.temperature_2m, weather.temperature_dewpoint,
#                                    output_dtypes=[np.float],
#                                    dask='parallelized'
#                                   )

# relative_humidity.name = 'relative_humidity'
# relative_humidity = relative_humidity.to_dataset()

In [20]:
# years, datasets = zip(*relative_humidity.groupby('time.year'))
# paths = [WEATHER_SRC / 'ecmwf' / 'daily_rel_humidity'/ f'rel_humidity_{y}.nc' for y in years]
# xr.save_mfdataset(datasets, paths)

## Generate WBT

> Similar to Relative Humidty, can skip if we directly calculate WBGT without saving intermediate data

In [5]:
# wbt = xr.apply_ufunc(climate_conversions.calculate_wbt_t_dew,
#                      weather.temperature_2m,  weather.temperature_dewpoint, weather.surface_pressure,
#                      output_dtypes=[np.dtype(weather.temperature_2m)],
#                      dask='parallelized'
#                     )

# wbt.name = 'wbt'
# wbt = wbt.to_dataset()

In [6]:
# years, datasets = zip(*wbt.groupby('time.year'))
# paths = [WEATHER_SRC / 'ecmwf' / 'daily_wbt'/ f'wbt_{y}.nc' for y in years]
# xr.save_mfdataset(datasets, paths)

## Generate WBGT max, mean

> Again, skip saving intermediate calculation of WBGT for every 6 hours

In [4]:
wbgt = xr.apply_ufunc(climate_conversions.calculate_wbgt_t_dew,
                     weather.temperature_2m,  weather.temperature_dewpoint, weather.surface_pressure,
                     output_dtypes=[np.dtype(weather.temperature_2m)],
                     dask='parallelized'
                    )

wbgt.name = 'wbgt'
wbgt = wbgt.to_dataset()

In [6]:
# years, datasets = zip(*wbgt.groupby('time.year'))
# paths = [WEATHER_SRC / 'ecmwf' / 'test'/ f'wbgt_{y}.nc' for y in years]
# xr.save_mfdataset(datasets, paths)


In [5]:
wbgt_mean = wbgt.resample(time='1D').mean()

/Users/jonathanchambers/miniconda3/envs/lancet/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.resample() has been modified to defer calculations. Instead of passing 'dim' and 'how="mean", instead consider using .resample(time="1D").mean() 
  if __name__ == '__main__':


In [7]:
years, datasets = zip(*wbgt_mean.groupby('time.year'))
paths = [WEATHER_SRC / 'ecmwf' / 'daily_wbgtmean'/ f'wbgt_mean_{y}.nc' for y in years]
xr.save_mfdataset(datasets, paths)

In [8]:
wbgt_max = wbgt.resample(time='1D').max()

In [9]:
years, datasets = zip(*wbgt_max.groupby('time.year'))
paths = [WEATHER_SRC / 'ecmwf' / 'daily_wbgtmax'/ f'wbgt_max_{y}.nc' for y in years]
xr.save_mfdataset(datasets, paths)

## Calculate WBGThalf

WBGThalf = the half way point between WBGTmax and WBGTmean.

In [5]:
wbgt_mean = xr.open_mfdataset(
    str(WEATHER_SRC / 'ecmwf' / 'daily_wbgtmean'/ 'wbgt_mean_*.nc'),
    chunks={'time':100}
                             )
wbgt_max = xr.open_mfdataset(
    str(WEATHER_SRC / 'ecmwf' / 'daily_wbgtmax'/ 'wbgt_max_*.nc'),
    chunks={'time':100}
)

In [7]:
wbgt_half = wbgt_mean + (wbgt_max - wbgt_mean) / 2

In [8]:
years, datasets = zip(*wbgt_half.groupby('time.year'))
paths = [WEATHER_SRC / 'ecmwf' / 'daily_wbgthalf'/ f'wbgt_half_{y}.nc' for y in years]
xr.save_mfdataset(datasets, paths)

# TODO more accurate method of calculating WBGT


In [ ]:
def fWBGTi(T_air, T_dew):
    # First step Psychrometric Wet bulb temperature calculation
    if T_dew > T_air:
        fWBGTi = T_air
        return fWBGTi

    # Iteration to get Tw from Td       
    Tw = T_dew
    Diff = 10000
    Ed = 0.6106 * np.exp(17.27 * T_dew / (237.3 + T_dew))
    while np.abs(Diff) + np.abs(Diffold) == np.abs(Diff + Diffold):
        Diffold = Diff
        Ew = 0.6106 * np.exp(17.27 * Tw / (237.3 + Tw))
        Diff = 1556 * Ed + 101 * Ta - 1556 * Ew + 1.484 * Ew * Tw - 1.484 * Ed * Tw - 101 * Tw
        Tw = Tw + 0.2
        if Tw > T_air:
            break
        
        if Tw > T_dew + 0.2:
            Tw = Tw - 0.2
        else:
            Tw = T_dew
    # Use Bernard formula to calculate natural wet bulb temperature from psychrometric wet bulb temperature for 1m/s wind speed
    fWBGTi = 0.67 * Tw + 0.33 * T_air
    return fWBGTi